In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Lambda
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
filename = "wonderland.txt"
all_text = open("data/shakespeare.txt").read().lower()
all_text = ''.join([i for i in all_text if not i.isdigit()])
SYMBOL_LIST = {'?', '.',  ')', ',', '!', ';', '(', ':'}
sonnets = all_text.replace('                   ', '').split('\n\n\n')
sonnets = [sonnet[1:] for sonnet in sonnets]
sonnets = [sonnet.replace('\n', ' ') for sonnet in sonnets]
sonnets = [sonnet.replace('  ', '') for sonnet in sonnets]
sonnets = [''.join([i for i in sonnet if not i in SYMBOL_LIST]) for sonnet in sonnets]

all_chars = sorted(list(set(''.join(sonnets))))
max_char = len(all_chars) - 1
char_map = {}
int_to_char = {}
int_to_arr = {}
for i in range(len(all_chars)):
    char_map[all_chars[i]] = i
    int_to_char[i] = all_chars[i]
    arr = [0] * len(all_chars) 
    arr[i] = 1
    int_to_arr[i] = arr

# print(sonnets)

X_train = []
Y_train = []

for sonnet in sonnets:
    start = 0
    end = 40
    while end < len(sonnet):
        sequence = sonnet[start:end]
        X_train.append([(char_map[c]) for c in sequence])
        Y_train.append(char_map[sonnet[end]])
        start += 1
        end += 1
        
samples = len(X_train)
timesteps = 40
features = len(all_chars)
X_train = [[int_to_arr[x] for x in seq] for seq in X_train]

X_train = np.array(X_train) 
X_train = np.reshape(X_train, (samples, timesteps, features))
Y_train = np.array([int_to_arr[y] for y in Y_train])


print(samples)

84692


In [3]:
# LSTM Layers:
temp = 0.25

model = Sequential()
model.add(LSTM(200, input_shape=(timesteps, features), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Lambda(lambda x: x / temp))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, Y_train, epochs=50, batch_size=128)


    

Epoch 1/50
84692/84692 [==============================] - 376s 4ms/step - loss: 2.3525
Epoch 2/50
84692/84692 [==============================] - 365s 4ms/step - loss: 1.9239
Epoch 3/50
84692/84692 [==============================] - 369s 4ms/step - loss: 1.7821
Epoch 4/50
84692/84692 [==============================] - 368s 4ms/step - loss: 1.6929
Epoch 5/50
84692/84692 [==============================] - 365s 4ms/step - loss: 1.6232
Epoch 6/50
84692/84692 [==============================] - 364s 4ms/step - loss: 1.5637
Epoch 7/50
84692/84692 [==============================] - 364s 4ms/step - loss: 1.5115
Epoch 8/50
84692/84692 [==============================] - 364s 4ms/step - loss: 1.4638
Epoch 9/50
84692/84692 [==============================] - 365s 4ms/step - loss: 1.4190
Epoch 10/50
84692/84692 [==============================] - 368s 4ms/step - loss: 1.3770
Epoch 11/50
84692/84692 [==============================] - 364s 4ms/step - loss: 1.3317
Epoch 12/50
84692/84692 [================

In [7]:
def next_char(input_str, m):
    X_in = np.array([[int_to_arr[char_map[c]] for c in input_str]])
    [probs] = m.predict(X_in, verbose=0)
    options = list(range(len(probs)))

    
    prediction = np.random.choice(options,p =  probs)
    
    return int_to_char[prediction]

result = "shall i compare thee to a summer's day t"
window = "shall i compare thee to a summer's day t"
for i in range(1000):
    prediction = next_char(window, model)
    result = result + prediction
    window = window[1:] + prediction

print(result)

shall i compare thee to a summer's day thou hast before decay which have no prouded confound are you worse when it hath in my life doth live thou usure in thy beauty's form in un of the laining sweets and death and statute of me for thou not farther frame that thou mayst true love contain and beauty looking on darks are men your sight and praise deceared this toe on thee and ghat's in heaven's sun at pution blunk dis rink to each that virtue o all the time contorded from the fairest rove is think all in in makes gracious in me but which full brow if thou should fasther frame for did chance write show my self will be this leave which i behold the lique and kind of present and do the wind whilst thou canst not to be a secold flowers in odour quick for i have now wend he way to that you so for my soul and ap is hide my moan then lost steal stay of love i think on thee when i am black any beauty's salse 'tis the wising he of a tropthere is by but of to that afford no eceive thy body's true

In [8]:
def get_poem(t, words_per_line, num_lines):
    result = []

    start = 0
    end = -1
    space_count = 0

    for i in range(len(t)):
        end += 1
        if t[i] == ' ':
            space_count += 1
        if space_count > words_per_line - 1:
            result.append(t[start:end])
            start = end + 1
            space_count = 0
        if i == len(t) - 1:
            result.append(t[start: end + 1])
        if len(result) == num_lines:
            break
    return result

In [9]:
poem = get_poem(result, 8, 10)
for i in range(len(poem)):
    if i % 2 == 0:
        print(poem[i] + ',')
    else:
        print(poem[i] + '.')

shall i compare thee to a summer's day,
thou hast before decay which have no prouded.
confound are you worse when it hath in,
my life doth live thou usure in thy.
beauty's form in un of the laining sweets,
and death and statute of me for thou.
not farther frame that thou mayst true love,
contain and beauty looking on darks are men.
your sight and praise deceared this toe on,
thee and ghat's in heaven's sun at pution.
